## Text Rank with NLTK

In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
from nltk.tokenize import sent_tokenize
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx

In [2]:
def get_only_text(url):
    """ 
    return the title and the text of the article
    at the specified url
    """
    page = urlopen(url)
    soup = BeautifulSoup(page, "lxml")
    text = ' '.join(map(lambda p: p.text, soup.find_all('p')))
  
    print ("=====================")
    print (text)
    print ("=====================")
 
    return soup.title.text, text    
 
     
url="https://www.whitehouse.gov/briefings-statements/remarks-president-trump-state-union-address-2/"
text = get_only_text(url)


				Remarks			 
Issued on:
February 6, 2019
 February 5, 2019
9:07 P.M. EST THE PRESIDENT:  Madam Speaker, Mr. Vice President, Members of Congress, the First Lady of the United States — (applause) — and my fellow Americans: We meet tonight at a moment of unlimited potential.  As we begin a new Congress, I stand here ready to work with you to achieve historic breakthroughs for all Americans. Millions of our fellow citizens are watching us now, gathered in this great chamber, hoping that we will govern not as two parties but as one nation.  (Applause.) The agenda I will lay out this evening is not a Republican agenda or a Democrat agenda.  It’s the agenda of the American people. Many of us have campaigned on the same core promises: to defend American jobs and demand fair trade for American workers; to rebuild and revitalize our nation’s infrastructure; to reduce the price of healthcare and prescription drugs; to create an immigration system that is safe, lawful, modern, and secure; and 

In [6]:
sentences = []
for s in text:
    sentences.append(sent_tokenize(s))

sentences = [y for x in sentences for y in x]
sentences[30:40]

['An amazing quality of life for all of our citizens is within reach.',
 'We can make our communities safer, our families stronger, our culture richer, our faith deeper, and our middle class bigger and more prosperous than ever before.',
 '(Applause.)',
 'But we must reject the politics of revenge, resistance, and retribution, and embrace the boundless potential of cooperation, compromise, and the common good.',
 '(Applause.)',
 'Together, we can break decades of political stalemate.',
 'We can bridge old divisions, heal old wounds, build new coalitions, forge new solutions, and unlock the extraordinary promise of America’s future.',
 'The decision is ours to make.',
 'We must choose between greatness or gridlock, results or resistance, vision or vengeance, incredible progress or pointless destruction.',
 'Tonight, I ask you to choose greatness.']

In [7]:
# Extract word vectors
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [8]:
len(word_embeddings)

400000

Text cleanining

In [9]:
# remove punctuations, numbers and special characters
clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")

# change to lowercase
clean_sentences = [s.lower() for s in clean_sentences]
stop_words = stopwords.words('english')
# function to remove stopwords
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

In [10]:
sentence_vectors = []
for i in clean_sentences:
    if len(i) != 0:
        v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
    else:
        v = np.zeros((100,))
    sentence_vectors.append(v)

In [11]:
# Create an empty similarity matrix
sim_mat = np.zeros([len(sentences), len(sentences)])

In [12]:
for i in range(len(sentences)):
    for j in range(len(sentences)):
        if i != j:
            sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]

In [19]:
nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
# Extract top 15 sentences as the summary representation
for i in range(10):
    print(ranked_sentences[i][1])

After 24 months of rapid progress, our economy is the envy of the world, our military is the most powerful on Earth, by far, and America — (applause) — America is again winning each and every day.
Millions of our fellow citizens are watching us now, gathered in this great chamber, hoping that we will govern not as two parties but as one nation.
I hope you can pass the USMCA into law so that we can bring back our manufacturing jobs in even greater numbers, expand American agriculture, protect intellectual property, and ensure that more cars are proudly stamped with our four beautiful words: “Made in the USA.”  (Applause.)
When Grace completed treatment last fall, her doctors and nurses cheered — they loved her; they still love her — with tears in their eyes as she hung up a poster that read: “Last day of chemo.”  (Applause.)
We do not know whether we will achieve an agreement, but we do know that, after two decades of war, the hour has come to at least try for peace.
Perhaps — (applause

## Gensim

In [21]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
from gensim.summarization import summarize
from gensim.summarization import keywords
import urllib.request
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests

In [22]:
text = requests.get('https://www.whitehouse.gov/briefings-statements/remarks-president-trump-state-union-address-2/').text

print('Summary:')
print(summarize(text, ratio=0.01))

print('\nKeywords:')
print(keywords(text, ratio=0.01))

2019-02-07 00:05:05,197 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2019-02-07 00:05:05,197 : INFO : built Dictionary(1371 unique tokens: ['bno', 'classnam', 'document', 'documentel', 'replac']...) from 539 documents (total 3371 corpus positions)


Summary:
<meta property="og:description" content="February 5, 2019 9:07 P.M. EST THE PRESIDENT:  Madam Speaker, Mr. Vice President, Members of Congress, the First Lady of the United States &#8212; (applause) &#8212; and my fellow Americans: We meet tonight at a moment of unlimited potential.
<meta name="twitter:description" content="February 5, 2019 9:07 P.M. EST THE PRESIDENT:  Madam Speaker, Mr. Vice President, Members of Congress, the First Lady of the United States &#8212; (applause) &#8212; and my fellow Americans: We meet tonight at a moment of unlimited potential.
<script type='application/ld+json'>{"@context":"http:\/\/schema.org","@type":"Organization","url":"https:\/\/www.whitehouse.gov\/briefings-statements\/remarks-president-trump-state-union-address-2\/","sameAs":["https:\/\/facebook.com\/whitehouse","https:\/\/instagram.com\/whitehouse","https:\/\/www.youtube.com\/user\/whitehouse","https:\/\/twitter.com\/whitehouse"],"@id":"#organization","name":"The White House","logo":

## Sumy

In [1]:
from __future__ import absolute_import
from __future__ import division, print_function, unicode_literals
 
from sumy.parsers.html import HtmlParser
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words
 
from sumy.summarizers.luhn import LuhnSummarizer
from sumy.summarizers.edmundson import EdmundsonSummarizer
from sumy.summarizers.lex_rank import LexRankSummarizer

In [2]:
LANGUAGE = "english"
SENTENCES_COUNT = 10

In [3]:
url="https://www.whitehouse.gov/briefings-statements/remarks-president-trump-state-union-address-2/"
parser = HtmlParser.from_url(url, Tokenizer(LANGUAGE))
print ("--LsaSummarizer--")    
summarizer = LsaSummarizer()
summarizer = LsaSummarizer(Stemmer(LANGUAGE))
summarizer.stop_words = get_stop_words(LANGUAGE)
for sentence in summarizer(parser.document, SENTENCES_COUNT):
    print(sentence)

--LsaSummarizer--
Now is the time for Congress to show the world that America is committed to ending illegal immigration and putting the ruthless coyotes, cartels, drug dealers, and human traffickers out of business.
Meanwhile, working-class Americans are left to pay the price for mass illegal migration: reduced jobs, lower wages, overburdened schools, hospitals that are so crowded you can’t get in, increased crime, and a depleted social safety net.
Tens of thousands of innocent Americans are killed by lethal drugs that cross our border and flood into our cities, including meth, heroin, cocaine, and fentanyl.
In the last two years, our brave ICE officers made 266,000 arrests of criminal aliens, including those charged or convicted of nearly 100,000 assaults, 30,000 sex crimes, and 4,000 killings or murders.
I hope you can pass the USMCA into law so that we can bring back our manufacturing jobs in even greater numbers, expand American agriculture, protect intellectual property, and ensu

In [33]:
print ("--LuhnSummarizer--")     
summarizer = LuhnSummarizer() 
summarizer = LuhnSummarizer(Stemmer(LANGUAGE))
summarizer.stop_words = ("I", "am", "the", "you", "are", "me", "is", "than", "that", "this")
for sentence in summarizer(parser.document, SENTENCES_COUNT):
    print(sentence)

--LuhnSummarizer--
Many of us have campaigned on the same core promises: to defend American jobs and demand fair trade for American workers; to rebuild and revitalize our nation’s infrastructure; to reduce the price of healthcare and prescription drugs; to create an immigration system that is safe, lawful, modern, and secure; and to pursue a foreign policy that puts America’s interests first.
Human traffickers and sex traffickers take advantage of the wide-open areas between our ports of entry to smuggle thousands of young girls and women into the United States and to sell them into prostitution and modern-day slavery.
Elvin says that, “If I can make sure these young girls get their justice, I’ve [really] done my job.” Thanks to his work, and that of his incredible colleagues, more than 300 women and girls have been rescued from the horror of this terrible situation, and more than 1,500 sadistic traffickers have been put behind bars.
The border city of El Paso, Texas used to have extre

In [5]:
print ("--EdmundsonSummarizer--")     
summarizer = EdmundsonSummarizer() 
words1 = ("economy", "fight", "trade", "china")
summarizer.bonus_words = words1
     
words2 = ("another", "and", "some", "next")
summarizer.stigma_words = words2
    
words3 = ("another", "and", "some", "next")
summarizer.null_words = words3
for sentence in summarizer(parser.document, SENTENCES_COUNT):
    print(sentence)

--EdmundsonSummarizer--
February 5, 2019
9:07 P.M. EST
We meet tonight at a moment of unlimited potential.
As we begin a new Congress, I stand here ready to work with you to achieve historic breakthroughs for all Americans.
Millions of our fellow citizens are watching us now, gathered in this great chamber, hoping that we will govern not as two parties but as one nation.
(Applause.)
The agenda I will lay out this evening is not a Republican agenda or a Democrat agenda.
It’s the agenda of the American people.
There is a new opportunity in American politics, if only we have the courage, together, to seize it.
Tonight, I am also asking you to join me in another fight that all Americans can get behind: the fight against childhood cancer.
END 10:29 P.M. EST


In [31]:
print ("--LexRankSummarizer--")   
summarizer = LexRankSummarizer()
summarizer = LexRankSummarizer(Stemmer(LANGUAGE))
summarizer.stop_words = ("I", "am", "the", "you", "are", "me", "is", "than", "that", "this")
for sentence in summarizer(parser.document, SENTENCES_COUNT):
    print(sentence)

--LexRankSummarizer--
Many of us have campaigned on the same core promises: to defend American jobs and demand fair trade for American workers; to rebuild and revitalize our nation’s infrastructure; to reduce the price of healthcare and prescription drugs; to create an immigration system that is safe, lawful, modern, and secure; and to pursue a foreign policy that puts America’s interests first.
Tonight, I am asking you to defend our very dangerous southern border out of love and devotion to our fellow citizens and to our country.
And thank you for being here.
When Elvin — (applause) — thank you.
To build on — (applause) — thank you.
I have great respect for President Xi, and we are now working on a new trade deal with China.
But we must do more.
As we make progress in these negotiations, we will be able to reduce our troop’s presence and focus on counterterrorism.
It’s time.
It’s the Americans!” (Applause.)
